In [1]:
print("Hello world")

Hello world


In [12]:
import numpy as np
import matplotlib as plt
import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

import IPythonMagic

from Timer import Timer

In [13]:
#Initialize our logging

%setup_logging

Global logger already initialized!


In [14]:
%cuda_context_handler context

Registering context in user workspace
Context already registered! Ignoring


In [43]:
kernel_sourcecode = """ 

__global__ void shmemReduction(float* output, float* input, int size){
    // First we stride through global memory and compute the maximum for every thread
    // gid = globalId
    int gid = blockIdx.x * blockDim.x + threadIdx.x; // blockIdx is always 0 because we use just one block
   
   float max_thread_value = -99999.99; //FixME : use proper value here
   
   float max_value;
    
    for (int i = threadIdx.x; i < size; i = i + blockDim.x){
    
        max_thread_value = fmaxf(max_value, input[i]); // I compute maximum value for every thread
        
    }
    
    // Temporary write to memory to check if thigns work so far
    
    output[threadIdx.x] = max_thread_value; 
    
    // Stored the pre-Thread maximum in shared memory
    
    __shared__ float max_shared[128];
    
    max_shared[threadIdx.x] = max_thread_value;
    
    // Sinchronize so to that all thread see the same memory
    
    __syncthreads();
    
    // Find the maximum in shared memory
    
    if (threadIdx.x < 64){
    
        // I have 2 warps that goes from 0 to 64 (ThreadIdx.x > 32). I need to have a sincthreads after this iteration
    
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 64]); // Reduce from 128 to 64 elements
        
    }
    
    __syncthreads();
    
    
    if (threadIdx.x < 32){
    
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 32]); // Reduce from 64 to 32 elements
        
    }
    
    if (threadIdx.x < 16){
    
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]); // Reduce from 32 to 16 elements
        
    }
     if (threadIdx.x < 8){
    
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 8]); // Reduce from 16 to 8 elements
        
    }
    
     if (threadIdx.x < 4){
    
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 4]); // Reduce from 8 to 4 elements
        
    }
    
     if (threadIdx.x < 2){
    
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 2]); // Reduce from 4 to 2 elements
        
    }
    
     if (threadIdx.x < 1){
    
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 1]); // Reduce from 2 to 1 elements
        
    }
    
    
    // Finally write out output

    if (threadIdx.x == 0){
    
        output[0] = max_shared[0];
    }
    
}

"""

kernel_module = cuda_compiler.SourceModule(kernel_sourcecode)
kernel_function = kernel_module.get_function("shmemReduction")

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:92: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(7): warning: variable "gid" was declared but never referenced




In [46]:
n = 256

a = np.random.random((1,n)).astype(np.float32)

print(a)

a_gpu = GPUArray(a.shape, a.dtype)

a_gpu.set(a)

num_threads = 128

# I start with an array 'cause at first I'm not finding maximum of all elements

b = np.empty((num_threads)).astype(np.float32)
b_gpu = GPUArray(b.shape, b.dtype)

c = np.empty(1).astype(np.float32)
c_gpu = GPUArray(c.shape, b.dtype)



[[0.5640829  0.46394816 0.2938098  0.5588757  0.75980484 0.06264941
  0.3925978  0.60767055 0.3309737  0.57635224 0.7923911  0.55936086
  0.8041371  0.67112637 0.38625455 0.9260951  0.5862123  0.7016446
  0.10683755 0.6505731  0.34600237 0.8908103  0.05418143 0.07550639
  0.5772979  0.5965145  0.098862   0.8790986  0.97726893 0.4068067
  0.55555344 0.73719335 0.72441036 0.65459996 0.56800693 0.6364083
  0.10276268 0.6301994  0.08153899 0.98833483 0.3884874  0.7305744
  0.15090898 0.4991452  0.9408307  0.7553358  0.25009578 0.00381485
  0.49507514 0.15268004 0.78813046 0.16401848 0.29689893 0.43173113
  0.20228119 0.646687   0.06284728 0.6491352  0.02166346 0.72434115
  0.70255506 0.07787921 0.18926497 0.84981555 0.5543002  0.41498616
  0.06682346 0.8694998  0.71777725 0.87532103 0.01027563 0.9601985
  0.9111745  0.8159941  0.5331719  0.21998025 0.3731966  0.8336696
  0.41300574 0.03510052 0.26774752 0.90999687 0.6833242  0.5786694
  0.36977598 0.6233644  0.5213112  0.55293244 0.6281193

In [47]:
block_size = (num_threads, 1, 1)
grid_size = (1, 1, 1)

kernel_function(c_gpu, a_gpu, np.int32(n), grid = grid_size, block = block_size)

c_gpu.get(c)

print(a)
print(c)
print(np.max(a))

[[0.5640829  0.46394816 0.2938098  0.5588757  0.75980484 0.06264941
  0.3925978  0.60767055 0.3309737  0.57635224 0.7923911  0.55936086
  0.8041371  0.67112637 0.38625455 0.9260951  0.5862123  0.7016446
  0.10683755 0.6505731  0.34600237 0.8908103  0.05418143 0.07550639
  0.5772979  0.5965145  0.098862   0.8790986  0.97726893 0.4068067
  0.55555344 0.73719335 0.72441036 0.65459996 0.56800693 0.6364083
  0.10276268 0.6301994  0.08153899 0.98833483 0.3884874  0.7305744
  0.15090898 0.4991452  0.9408307  0.7553358  0.25009578 0.00381485
  0.49507514 0.15268004 0.78813046 0.16401848 0.29689893 0.43173113
  0.20228119 0.646687   0.06284728 0.6491352  0.02166346 0.72434115
  0.70255506 0.07787921 0.18926497 0.84981555 0.5543002  0.41498616
  0.06682346 0.8694998  0.71777725 0.87532103 0.01027563 0.9601985
  0.9111745  0.8159941  0.5331719  0.21998025 0.3731966  0.8336696
  0.41300574 0.03510052 0.26774752 0.90999687 0.6833242  0.5786694
  0.36977598 0.6233644  0.5213112  0.55293244 0.6281193